In [1]:
# Jhose Immanuel Sembiring
# 1103202047
# TK44G4
# Modern-CNN_ResNeXt

#Library

In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Blok ResNeXt

In [3]:
class ResNeXtBlock(nn.Module):
    def __init__(self, in_channels, out_channels, cardinality, stride=1):
        super(ResNeXtBlock, self).__init__()
        group_width = out_channels // cardinality
        self.conv1 = nn.Conv2d(in_channels, group_width * cardinality, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(group_width * cardinality)
        self.conv2 = nn.Conv2d(group_width * cardinality, group_width * cardinality, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(group_width * cardinality)
        self.conv3 = nn.Conv2d(group_width * cardinality, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.downsample = None

    def forward(self, x):
        identity = x

        out = self.conv1(x)  # First convolution layer
        out = self.bn1(out)  # Batch normalization
        out = self.relu(out) # ReLU activation

        out = self.conv2(out)  # Second convolution layer
        out = self.bn2(out)    # Batch normalization
        out = self.relu(out)   # ReLU activation

        out = self.conv3(out)  # Third convolution layer
        out = self.bn3(out)    # Batch normalization

        if self.downsample is not None:
            identity = self.downsample(x)  # Adjust dimensions if needed

        out += identity  # Add the input (identity)
        out = self.relu(out)  # ReLU activation

        return out


#Definisi model ResNeXt

In [4]:
class ResNeXt(nn.Module):
    def __init__(self, block, layers, cardinality=8, num_classes=10):
        super(ResNeXt, self).__init__()
        self.in_channels = 64
        self.cardinality = cardinality

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 128, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 256, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, self.cardinality, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, self.cardinality))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Define the smaller ResNeXt model
def ResNeXt29_8x4d():
    return ResNeXt(ResNeXtBlock, [1, 1, 1], cardinality=8)

model = ResNeXt29_8x4d().to(device)


#Mempersiapkan Dataset

In [5]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(64),  # Smaller crop size
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:02<00:00, 67914010.23it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#Melatih Model

In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Training function
def train(epoch):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

# Evaluation function
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Train and evaluate the model
for epoch in range(5):  # Train for 5 epochs
    train(epoch)
    test()


[Epoch 1, Batch 100] loss: 2.149
[Epoch 1, Batch 200] loss: 2.032
[Epoch 1, Batch 300] loss: 1.929
[Epoch 1, Batch 400] loss: 1.829
[Epoch 1, Batch 500] loss: 1.789
[Epoch 1, Batch 600] loss: 1.769
[Epoch 1, Batch 700] loss: 1.750
[Epoch 1, Batch 800] loss: 1.703
[Epoch 1, Batch 900] loss: 1.671
[Epoch 1, Batch 1000] loss: 1.648
[Epoch 1, Batch 1100] loss: 1.644
[Epoch 1, Batch 1200] loss: 1.569
[Epoch 1, Batch 1300] loss: 1.603
[Epoch 1, Batch 1400] loss: 1.578
[Epoch 1, Batch 1500] loss: 1.563
Accuracy of the network on the 10000 test images: 42.28%
[Epoch 2, Batch 100] loss: 1.551
[Epoch 2, Batch 200] loss: 1.518
[Epoch 2, Batch 300] loss: 1.527
[Epoch 2, Batch 400] loss: 1.483
[Epoch 2, Batch 500] loss: 1.523
[Epoch 2, Batch 600] loss: 1.490
[Epoch 2, Batch 700] loss: 1.467
[Epoch 2, Batch 800] loss: 1.474
[Epoch 2, Batch 900] loss: 1.459
[Epoch 2, Batch 1000] loss: 1.456
[Epoch 2, Batch 1100] loss: 1.457
[Epoch 2, Batch 1200] loss: 1.393
[Epoch 2, Batch 1300] loss: 1.413
[Epoch 2,

#Visualisasi Hasil

In [ ]:
# Initialize lists to store loss and accuracy
train_losses = []
test_accuracies = []

# Modified training function to track loss
def train_with_tracking(epoch):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            train_losses.append(running_loss / 100)
            running_loss = 0.0

# Modified evaluation function to track accuracy
def test_with_tracking():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    test_accuracies.append(accuracy)
    print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

# Train and evaluate the model with tracking
for epoch in range(5):  # Train for 5 epochs
    train_with_tracking(epoch)
    test_with_tracking()

# Plot training loss and test accuracy
epochs = range(1, 6)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'o-', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, test_accuracies, 'o-', label='Test accuracy')
plt.title('Test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()


[Epoch 1, Batch 100] loss: 1.133
[Epoch 1, Batch 200] loss: 1.153
[Epoch 1, Batch 300] loss: 1.142
[Epoch 1, Batch 400] loss: 1.134
[Epoch 1, Batch 500] loss: 1.170
[Epoch 1, Batch 600] loss: 1.087
[Epoch 1, Batch 700] loss: 1.124
[Epoch 1, Batch 800] loss: 1.149
[Epoch 1, Batch 900] loss: 1.127
[Epoch 1, Batch 1000] loss: 1.138
[Epoch 1, Batch 1100] loss: 1.145
